In [1]:
from datascience import *
from datascience.predicates import are
path_data = '../../../../data/'
import numpy as np
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from urllib.request import urlopen
import re
from bancor3_simulator.protocol import BancorV3
from bancor3_simulator.core.settings import GlobalSettings
from decimal import Decimal
from fractions import Fraction
import pandas as pd

# Recall from an earlier chapter that we already defined the cooldown period and the exit fees as follows.
cooldown_time: int = 7
withdrawal_fee: Decimal = Decimal('0.0025')
whitelisted_tokens: list = ['bnt', 'eth', 'wbtc', 'link']
bnt_funding_limit: Decimal = Decimal('40000')
bnt_min_liquidity: Decimal = Decimal('10000')
price_feeds = pd.DataFrame({'INDX':[0],
                            'bnt':[2.5],
                            'link':[15.00],
                            'eth':[2500.00],
                            'wbtc':[40000.00]})

network_fee: Decimal = Decimal('0.2')
trading_fee: Decimal = Decimal('0.01')

# There are other possible configuration settings available, however for the present purpose we will use the defaults.
v3 = BancorV3(whitelisted_tokens=whitelisted_tokens,
              network_fee=network_fee,
              trading_fee=trading_fee,
              cooldown_time=cooldown_time,
              withdrawal_fee=withdrawal_fee,
              bnt_min_liquidity=bnt_min_liquidity,
              bnt_funding_limit=bnt_funding_limit,
              price_feeds=price_feeds)

v3.create_user('Alice')
v3.create_user('Bob')
v3.create_user('Charlie')

# The actual deposits.

v3.stake(tkn_amt=100, tkn_name='eth', user_name='Alice')

v3.stake(tkn_amt=10000, tkn_name='link', user_name='Charlie')

v3.stake(tkn_amt=100, tkn_name='wbtc', user_name='Bob')
#
v3.stake(tkn_amt=1, tkn_name='eth', user_name='Alice')

v3.stake(tkn_amt=1, tkn_name='link', user_name='Charlie')

v3.stake(tkn_amt=1, tkn_name='wbtc', user_name='Bob')

v3.dao_msig_init_pools(whitelisted_tokens)

v3.create_user('Trader')

v3.trade(swap_amount=2000, source_token='bnt',  target_token='link', user_name='Trader', timestep=0)

v3.trade(swap_amount=302.9981, source_token='link',  target_token='bnt', user_name='Trader', timestep=0)

v3.trade(swap_amount=1, source_token='eth',  target_token='wbtc', user_name='Trader', timestep=0)

def read_url(url): 
    return re.sub('\\s+', ' ', urlopen(url).read().decode())

# Staking TKN

In [2]:
v3.describe(decimals=4)

,Trading Liquidity,Vault,Staking,ERC20 Contracts,Vortex,External Protection,Protocol Wallet
0,,bnt=60022.3847,bnt=60023.5998,bnbnt=60000.0000,bnt=12.2692,bnt=0.0000,bnbnt=60000.0000
1,bnt=19055.2381 eth=21.0000,eth=102.0000,eth=101.0000,bneth=101.0000,,eth=0.0000,
2,bnt=20940.8835 wbtc=1.1943,wbtc=100.9443,wbtc=101.0005,bnwbtc=101.0000,,wbtc=0.0000,
3,bnt=20013.9939 link=3336.3314,link=10003.9981,link=10003.4242,bnlink=10001.0000,,link=0.0000,
4,,,,,,,


In [3]:
v3.describe(rates=True, decimals=4)

,index,0
0,bnt,"Spot Rate=0.0000, EMA Rate=0.0000"
1,eth,"Spot Rate=907.3923, EMA Rate=1000.0000"
2,wbtc,"Spot Rate=17534.2111, EMA Rate=16000.0000"
3,link,"Spot Rate=5.9988, EMA Rate=6.0000"


# Staking TKN

The ema serves as a detection mechanism for unusual spikes in the price of an asset against BNT, and prevents changes to the depth of the trading pools to protect the system from harm. To illustrate the effects of the ema, the current state of the system in the present narrative will be carried forward, with each of Alice, Bob, and Charlie providing one additional TKN to the system. Unlike the prior examples where liquidity is added, both the ema, and the pool token valuation need to be taken under consideration.

Before the demonstration continues, recall that each of the pools have a preset 4,000 BNT funding limit at this point in the narrative, and up to this point in time all three have only exhausted half of that amount. Further, recall that the ability to increase the available trading liquidity is determined by the vault balance of the TKN in question, and the current depth of the pool, rather than the amount being contributed by a user during any deposit event.

In [4]:
timestep = v3.global_state.tokens['wbtc'].timestep
print('timestep', timestep)
v3.stake(tkn_amt=1, tkn_name='eth',  user_name='Alice', timestep=timestep)

v3.describe(decimals=4)

timestep 0


,Trading Liquidity,Vault,Staking,ERC20 Contracts,Vortex,External Protection,Protocol Wallet
0,,bnt=60022.3847,bnt=60023.5998,bnbnt=60000.0000,bnt=12.2692,bnt=0.0000,bnbnt=60000.0000
1,bnt=19055.2381 eth=21.0000,eth=103.0000,eth=102.0000,bneth=102.0000,,eth=0.0000,
2,bnt=20940.8835 wbtc=1.1943,wbtc=100.9443,wbtc=101.0005,bnwbtc=101.0000,,wbtc=0.0000,
3,bnt=20013.9939 link=3336.3314,link=10003.9981,link=10003.4242,bnlink=10001.0000,,link=0.0000,
4,,,,,,,


In [5]:
v3.describe(rates=True, decimals=4)

,index,0
0,bnt,"Spot Rate=0.0000, EMA Rate=0.0000"
1,eth,"Spot Rate=907.3923, EMA Rate=1000.0000"
2,wbtc,"Spot Rate=17534.2111, EMA Rate=16000.0000"
3,link,"Spot Rate=5.9988, EMA Rate=6.0000"


Alice now deposits 1 ETH. Since no fees have yet accrued to the bnETH pool tokens (i.e. ETH has not yet been the target of any particular trade), the value of the bnETH pool token remains unchanged. Her 1 ETH is added to the vault, the staking ledger is updated, and a new pool token is created for Alice.

Assume that this deposit is occurring on a block wherein no actions have taken place on the ETH pool. Therefore, the ema has not been updated in this block, and it remains where it was. This allows for the effective price quoted by the pool to be evaluated with a view to detecting artificial price manipulation. The protocol's confidence is determined by the relative deviation between the ema and spot prices; the protocol will only allow the pool's trading liquidity to be changed if the difference between these values is small.

In this instance, the addition of Alice's 1 ETH occurs when the ema on ETH remains at 1,000, whereas its spot rate is 907.26. The deviation between the spot rate and the ema is measured relative to the ema, as follows:

Since Δ is fixed at 0.01, the equation above can be expressed as the following boolean:

![](https://lh5.googleusercontent.com/blsejE9xJDrmvWZPTX9pyIZnAHe6_FiCmwoKm-IryJ0w678oabuREMMxDo5PrZRbSg68QLAFxG1COySwZTquiD0rfdRxK63u4yGw8In-kcsQFYFuwJWP4JKXebJloN_mkTZ5deQ7YCEaTO5esA)\
where r is the spot rate. In this case, Δ is 0.0927; the tolerance of the protocol is 0.01. Therefore, the protocol will not allow the trading liquidity to be changed on this block. However, Alice's 1 ETH is still accepted into the vault, and she is issued bnETH pool tokens as normal. Since there is no change to the trading liquidity, the protocol will not mint BNT, and will not issue itself bnBNT.

Bob now deposits 1 wBTC. In contrast to Alice’s situation, fees have accumulated on the wBTC pool token. Therefore, the rate of pool token issuance is affected. The staking ledger is reporting a staked amount of wBTC of 101.00045, and the supply of bnwBTC is 101. Therefore, the rate of bnwBTC to wBTC is 0.999995543, and this quotient determines the number of bnwBTC pool tokens Bob will receive.

Assume that this deposit is occurring on the same block as Alice’s deposit, where the trade between ETH and wBTC had been executed on the prior block. Therefore, the wBTC ema has not been updated in this block, and also remains at the genesis rate.

In this instance, the addition of Bob’s 1 wBTC coccurs when the ema on wBTC remains at 16,000, whereas its spot rate is at 17,534. The deviation (Δ) between the spot rate and the ema is 0.0959, and as before, the protocol will not allow the trading liquidity to be changed on this block. Regardless, the vault still accepts Bob’s wBTC, and issues him bnwBTC as normal.


In [6]:
timestep = v3.global_state.tokens['wbtc'].timestep
print('timestep', timestep)
v3.stake(tkn_amt=1, tkn_name='wbtc', user_name='Bob', timestep=timestep)

v3.describe(decimals=4)

timestep 0


,Trading Liquidity,Vault,Staking,ERC20 Contracts,Vortex,External Protection,Protocol Wallet
0,,bnt=60022.3847,bnt=60023.5998,bnbnt=60000.0000,bnt=12.2692,bnt=0.0000,bnbnt=60000.0000
1,bnt=19055.2381 eth=21.0000,eth=103.0000,eth=102.0000,bneth=102.0000,,eth=0.0000,
2,bnt=20940.8835 wbtc=1.1943,wbtc=101.9443,wbtc=102.0005,bnwbtc=102.0000,,wbtc=0.0000,
3,bnt=20013.9939 link=3336.3314,link=10003.9981,link=10003.4242,bnlink=10001.0000,,link=0.0000,
4,,,,,,,


In [7]:
v3.describe(rates=True, decimals=4)

,index,0
0,bnt,"Spot Rate=0.0000, EMA Rate=0.0000"
1,eth,"Spot Rate=907.3923, EMA Rate=1000.0000"
2,wbtc,"Spot Rate=17534.2111, EMA Rate=16000.0000"
3,link,"Spot Rate=5.9988, EMA Rate=6.0000"


Charlie now deposits 1 LINK. As was the case for Bob, the fee accrual on bnLINK will affect the issuance of pool tokens. The staking ledger is reporting a staked amount of LINK of 10,003.42, and the supply of bnLINK is 10,001. Therefore, the rate of bnLINK to LINK is 0.999758, and the issuance of bnLINK to Charlie is determined by this number.

Assume that this deposit occurs on the block following the two LINK trades previously discussed. In this instance, the addition of Charlie’s 1 LINK occurs when the ema on LINK is 5.9998, whereas its spot rate is at 5.9988. The deviation (Δ) between the spot rate and the ema is 0.00017, which is inside the tolerance levels of the protocol, and the protocol will allow the trading liquidity to update.

Trading liquidity always updates according to a strict set of criteria. In simple terms, the protocol will always try to increase the liquidity by as much as possible, up to a maximum BNT liquidity increase of 2× relative to the current depth, and without exceeding the preset funding limit. In this example, the current BNT depth on the LINK pool is 20,013.993668. Therefore, with a sufficient funding limit, the pool could double in size, up to a maximum BNT depth of 40,027.98734 BNT. However, the protocol has previously contributed 20,000 BNT of its 40,000 BNT funding limit - it can only increase the trading liquidity by a maximum of 20,000 BNT. The protocol will always default to the smallest of these two values.

By convention, the protocol trusts the ema rate over the spot price. Therefore, in this example, the BNT/LINK quotient is treated as 5.9998. The maximum allowed increase is the BNT funding limit, 20,000 BNT. Therefore, 3333.998732 LINK and 20,000 BNT are added to the trading liquidity. As the protocol is minting BNT, it is simultaneously issuing new bnBNT pool tokens for itself. Since the bnBNT pool token has appreciated in value, this will need to be taken into account.

In [8]:
timestep = v3.global_state.tokens['wbtc'].timestep + 1
print('timestep', timestep)
v3.stake(tkn_amt=1, tkn_name='link', user_name='Charlie', timestep=timestep)

v3.describe(decimals=4)

timestep 1


,Trading Liquidity,Vault,Staking,ERC20 Contracts,Vortex,External Protection,Protocol Wallet
0,,bnt=80006.4040,bnt=80007.6190,bnbnt=79976.1621,bnt=12.2692,bnt=0.0000,bnbnt=59992.1428
1,bnt=19055.2381 eth=21.0000,eth=103.0000,eth=102.0000,bneth=102.0000,,eth=0.0000,
2,bnt=20940.8835 wbtc=1.1943,wbtc=101.9443,wbtc=102.0005,bnwbtc=102.0000,,wbtc=0.0000,
3,bnt=39998.0132 link=6666.3355,link=10004.9981,link=10004.4242,bnlink=10001.9998,,link=0.0000,
4,,,,,,,


The staking ledger is reporting a staked amount of BNT of 60,022.384, and the supply of bnBNT is 60,000. Therefore, the rate of bnBNT to BNT is 0.999627065. As the protocol mints 20,000 BNT to increase the trading liquidity, this amount is added to the BNT staking ledger, and the protocol issues itself 19,992.5413 bnBNT pool tokens.

In [9]:
v3.describe(decimals=4)

,Trading Liquidity,Vault,Staking,ERC20 Contracts,Vortex,External Protection,Protocol Wallet
0,,bnt=80006.4040,bnt=80007.6190,bnbnt=79976.1621,bnt=12.2692,bnt=0.0000,bnbnt=59992.1428
1,bnt=19055.2381 eth=21.0000,eth=103.0000,eth=102.0000,bneth=102.0000,,eth=0.0000,
2,bnt=20940.8835 wbtc=1.1943,wbtc=101.9443,wbtc=102.0005,bnwbtc=102.0000,,wbtc=0.0000,
3,bnt=39998.0132 link=6666.3355,link=10004.9981,link=10004.4242,bnlink=10001.9998,,link=0.0000,
4,,,,,,,
